##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TFX Python function component tutorial



Note: We recommend running this tutorial in a Colab notebook, with no setup
required!  Just click "Run in Google Colab".

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/python_function_component">
<img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/tfx/python_function_component.ipynb">
<img src="https://www.tensorflow.org/images/colab_logo_32px.png">Run in Google Colab</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/tfx/tree/master/docs/tutorials/tfx/python_function_component.ipynb">
<img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">View source on GitHub</a></td>
<td><a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/tfx/docs/tutorials/tfx/python_function_component.ipynb">
<img width=32px src="https://www.tensorflow.org/images/download_logo_32px.png">Download notebook</a></td>
</table></div>


This notebook contains an examples on how to author and run Python function
components within the TFX InteractiveContext and in a locally-orchestrated TFX
pipeline.

For more context and information, see the [Custom Python function components](https://www.tensorflow.org/tfx/guide/custom_function_component)
page on the TFX documentation site.

## Setup

We will first install TFX and import necessary modules. TFX requires Python 3.

### Check the system Python version


In [1]:
import sys
sys.version

'3.7.13 (default, Apr 24 2022, 01:04:09) \n[GCC 7.5.0]'

### Upgrade Pip

To avoid upgrading Pip in a system when running locally, check to make sure
that we're running in Colab.  Local systems can of course be upgraded
separately.

In [2]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 5.3 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


### Install TFX

**Note: In Google Colab, because of package updates, the first time you run
this cell you must restart the runtime (Runtime > Restart runtime ...).**

In [3]:
!pip install -U tfx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.6/206.6 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.0/147.0 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.3/435.3 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

## Did you restart the runtime?

If you are using Google Colab, the first time that you run the cell above, you
must restart the runtime (Runtime > Restart runtime ...). This is because of
the way that Colab loads packages.

### Import packages
We import TFX and check its version.

In [1]:
# Check version
from tfx import v1 as tfx
tfx.__version__

'1.8.0'

## Custom Python function components

In this section, we will create components from Python functions. We will notbe
doing any real ML problem — these simple functions are just used to illustrate
the Python function component development process.

See [Python function based component
guide](https://www.tensorflow.org/tfx/guide/custom_function_component)
for more documentation.

### Create Python custom components

We begin by writing a function that generate some dummy data. This is written
to its own Python module file.

In [2]:
%%writefile my_generator.py

import os
import tensorflow as tf  # Used for writing files.

from tfx import v1 as tfx

# Non-public APIs, just for showcase.
from tfx.types.experimental.simple_artifacts import Dataset

@tfx.dsl.components.component
def MyGenerator(data: tfx.dsl.components.OutputArtifact[Dataset]):
  """Create a file with dummy data in the output artifact."""
  with tf.io.gfile.GFile(os.path.join(data.uri, 'data_file.txt'), 'w') as f:
    f.write('Dummy data')

  # Set metadata and ensure that it gets passed to downstream components.
  data.set_string_custom_property('my_custom_field', 'my_custom_value')

Writing my_generator.py


Next, we write a second component that uses the dummy data produced.
We will just calculate hash of the data and return it.

In [3]:
%%writefile my_consumer.py

import hashlib
import os
import tensorflow as tf

from tfx import v1 as tfx

# Non-public APIs, just for showcase.
from tfx.types.experimental.simple_artifacts import Dataset
from tfx.types.standard_artifacts import String

@tfx.dsl.components.component
def MyConsumer(data: tfx.dsl.components.InputArtifact[Dataset],
               hash: tfx.dsl.components.OutputArtifact[String],
               algorithm: tfx.dsl.components.Parameter[str] = 'sha256'):
  """Reads the contents of data and calculate."""
  with tf.io.gfile.GFile(
      os.path.join(data.uri, 'data_file.txt'), 'r') as f:
    contents = f.read()
  h = hashlib.new(algorithm)
  h.update(tf.compat.as_bytes(contents))
  hash.value = h.hexdigest()

  # Read a custom property from the input artifact and set to the output.
  custom_value = data.get_string_custom_property('my_custom_field')
  hash.set_string_custom_property('input_custom_field', custom_value)

Writing my_consumer.py


### Run in-notebook with the InteractiveContext
Now, we will demonstrate usage of our new components in the TFX
InteractiveContext.

For more information on what you can do with the TFX notebook
InteractiveContext, see the in-notebook [TFX Keras Component Tutorial](https://www.tensorflow.org/tfx/tutorials/tfx/components_keras).

In [4]:
from my_generator import MyGenerator
from my_consumer import MyConsumer

#### Construct the InteractiveContext

In [5]:
# Here, we create an InteractiveContext using default parameters. This will
# use a temporary directory with an ephemeral ML Metadata database instance.
# To use your own pipeline root or database, the optional properties
# `pipeline_root` and `metadata_connection_config` may be passed to
# InteractiveContext. Calls to InteractiveContext are no-ops outside of the
# notebook.
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
context = InteractiveContext()

#### Run your component interactively with `context.run()`
Next, we run our components interactively within the notebook with
`context.run()`. Our consumer component uses the outputs of the generator
component.

In [6]:
generator = MyGenerator()
context.run(generator)

ExecutionResult(
    component_id: MyGenerator
    execution_id: 1
    outputs:
        data: OutputChannel(artifact_type=Dataset, producer_component_id=MyGenerator, output_key=data, additional_properties={}, additional_custom_properties={}))

In [7]:
consumer = MyConsumer(
    data=generator.outputs['data'],
    algorithm='md5')
context.run(consumer)

ExecutionResult(
    component_id: MyConsumer
    execution_id: 2
    outputs:
        hash: OutputChannel(artifact_type=String, producer_component_id=MyConsumer, output_key=hash, additional_properties={}, additional_custom_properties={}))

After execution, we can inspect the contents of the "hash" output artifact of
the consumer component on disk.

In [8]:
!tail -v {consumer.outputs['hash'].get()[0].uri}

==> /tmp/tfx-interactive-2022-06-25T11_52_37.606095-0oet8rp9/MyConsumer/hash/2/value <==
0015fe7975d1a2794b59aa12635703f1

That's it, and you've now written and executed your own custom components!

### Write a pipeline definition

Next, we will author a pipeline using these same components. While using the
`InteractiveContext` within a notebook works well for experimentation, defining
a pipeline lets you deploy your pipeline on local or remote runners for
production usage.

Here, we will demonstrate usage of the LocalDagRunner running locally on your
machine. For production execution, the Airflow or Kubeflow runners may
be more suitable.

#### Construct a pipeline

In [9]:
import os
import tempfile
from tfx import v1 as tfx

# Select a persistent TFX root directory to store your output artifacts.
# For demonstration purposes only, we use a temporary directory.
PIPELINE_ROOT = tempfile.mkdtemp()
# Select a pipeline name so that multiple runs of the same logical pipeline
# can be grouped.
PIPELINE_NAME = "function-based-pipeline"
# We use a ML Metadata configuration that uses a local SQLite database in
# the pipeline root directory. Other backends for ML Metadata are available
# for production usage.
METADATA_CONNECTION_CONFIG = tfx.orchestration.metadata.sqlite_metadata_connection_config(
    os.path.join(PIPELINE_ROOT, 'metadata.sqlite'))

def function_based_pipeline():
  # Here, we construct our generator and consumer components in the same way.
  generator = MyGenerator()
  consumer = MyConsumer(
      data=generator.outputs['data'],
      algorithm='md5')

  return tfx.dsl.Pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      components=[generator, consumer],
      metadata_connection_config=METADATA_CONNECTION_CONFIG)

my_pipeline = function_based_pipeline()

#### Run your pipeline with the `LocalDagRunner`

In [10]:
tfx.orchestration.LocalDagRunner().run(my_pipeline)

We can inspect the output artifacts generated by this pipeline execution.

In [11]:
!find {PIPELINE_ROOT}

/tmp/tmpyy7ry9_n
/tmp/tmpyy7ry9_n/MyConsumer
/tmp/tmpyy7ry9_n/MyConsumer/.system
/tmp/tmpyy7ry9_n/MyConsumer/.system/executor_execution
/tmp/tmpyy7ry9_n/MyConsumer/.system/executor_execution/2
/tmp/tmpyy7ry9_n/MyConsumer/hash
/tmp/tmpyy7ry9_n/MyConsumer/hash/2
/tmp/tmpyy7ry9_n/MyConsumer/hash/2/value
/tmp/tmpyy7ry9_n/metadata.sqlite
/tmp/tmpyy7ry9_n/MyGenerator
/tmp/tmpyy7ry9_n/MyGenerator/data
/tmp/tmpyy7ry9_n/MyGenerator/data/1
/tmp/tmpyy7ry9_n/MyGenerator/data/1/data_file.txt
/tmp/tmpyy7ry9_n/MyGenerator/.system
/tmp/tmpyy7ry9_n/MyGenerator/.system/executor_execution
/tmp/tmpyy7ry9_n/MyGenerator/.system/executor_execution/1


You have now written your own custom components and orchestrated their
execution on the LocalDagRunner! For next steps, check out additional tutorials
and guides on the [TFX website](https://www.tensorflow.org/tfx).